In [8]:
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import pandas as pd
import numpy as np
import sqlite3
import os

SRC_DIR = os.path.dirname(os.getcwd())

In [9]:
# QB data stored in the DB-comes from qb_etl.py
dbname = "quickbooks.db"
dbpath = os.path.join(SRC_DIR,"db",dbname)
conn = sqlite3.connect(dbpath) 
qbdf = pd.read_sql("select * from categorized_items", conn)

# Budget data read from static csv
budget_file = os.path.join(SRC_DIR,'config','qb_to_budget_map.csv')
budgetdf = pd.read_csv(budget_file)

In [11]:
# preprocessing/data manipulation
budget_items = budgetdf['QB_Item'].unique()

expenses = qbdf.loc[qbdf['Account_Type']=='Expenses']
income = qbdf.loc[qbdf['Account_Type'] == 'Income']

# check for unrecognized types and categories
types = qbdf["Transaction Type"].unique()
expected_types = ['Check','Expense','Deposit']
for t in types:
    if t not in expected_types:
        print(f"Warning: {t} not a recognized type")

# expenses not in a budget category will not show up in the bugdet bar plots
# BUT they will show up in profit/loss calculations
for item in expenses['item'].unique():
    if item not in budget_items:
        print(f"Warning: {item} not in any budget category")

In [15]:
# Combine expense items with budget items
item_totals = expenses.groupby('item').aggregate({"Amount":"sum","Date":'count'}).reset_index()
item_totals.columns = ['item','Amount','Transactions']
all_totals = pd.merge(budgetdf,item_totals, left_on='QB_Item', right_on="item", how = 'left')
subcategory_totals = all_totals.groupby("Subcategory").aggregate({"Budget":"sum","Amount":"sum"}).reset_index()

total_expense = expenses['Amount'].sum()
total_income = income['Amount'].sum()
net_profit = total_income - total_expense

In [24]:
subcategory_totals.columns

Index(['Subcategory', 'Budget', 'Amount'], dtype='object')

In [31]:
#fig, ax = plt.subplots(figsize = (10,6))
#ind = np.arange(len(subcategory_totals))
#width = 0.45

budget_bar = subcategory_totals.hvplot.bar(x='Subcategory',
                                           y=['Budget','Amount'],
                                          ylabel="Expenses",
                                          rot=90,
                                          width = 1000,
                                          legend="top_left").opts(multi_level=False)


tab = pn.panel(budget_bar)
pn.serve(tab)
#ax.set_ylabel("Monthly Expenses")
#ax.set_title("YTD Report", fontsize=20)
#midx = ind.mean()
#midy = max([subcategory_totals['Budget'].max(), subcategory_totals["Amount"].max()])*0.8
#color = 'green'
#if net_profit>0:
#    color = 'green'
#else:
#    color = 'red'
#ax.text(midx-1, midy, 'Income: {:.2f}\nExpenses: {:.2f}\nNet Profit: {:.2f}'.format(total_income,total_expense, net_profit), fontsize=14, bbox=dict(facecolor=color, alpha=0.5))
#ax.legend()

#ax.set_xticks(ind+width/2, subcategory_totals['Subcategory'], rotation=90, fontsize=15)
#plt.tight_layout()
#plt.savefig(f'{month_name}.png')

Launching server at http://localhost:58353
